In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._

// Initialize SparkSession
val spark = SparkSession.builder()
  .appName("User Rating History Partitioning")
  .getOrCreate()


spark = org.apache.spark.sql.SparkSession@7338133


org.apache.spark.sql.SparkSession@7338133

In [ ]:
val ratingsPath = "gs://bhargav-assignments/Day16And17Task/rating.csv"
val outputPath = "hdfs:///user/Day16AndDay17/case_study_2/user-data"

val ratingsDF = spark.read.option("header", "true").csv(ratingsPath)

ratingsPath = gs://bhargav-assignments/Day16And17Task/rating.csv
outputPath = hdfs:///user/Day16AndDay17/case_study_2/user-data
ratingsDF = [userId: string, movieId: string ... 2 more fields]


In [3]:
// Filter out invalid or incomplete records
val validRatingsDF = ratingsDF
  .filter(col("userId").isNotNull && col("movieId").isNotNull && col("rating").isNotNull)
  .withColumn("userId", col("userId").cast("int")) // Ensure userId is integer
  .withColumn("movieId", col("movieId").cast("int")) // Ensure movieId is integer
  .withColumn("rating", col("rating").cast("float")) // Ensure rating is float

// Show valid ratings
println("Valid Ratings DataFrame:")
validRatingsDF.show(10, truncate = false)


Valid Ratings DataFrame:
+------+-------+------+-------------------+
|userId|movieId|rating|timestamp          |
+------+-------+------+-------------------+
|1     |2      |3.5   |2005-04-02 23:53:47|
|1     |29     |3.5   |2005-04-02 23:31:16|
|1     |32     |3.5   |2005-04-02 23:33:39|
|1     |47     |3.5   |2005-04-02 23:32:07|
|1     |50     |3.5   |2005-04-02 23:29:40|
|1     |112    |3.5   |2004-09-10 03:09:00|
|1     |151    |4.0   |2004-09-10 03:08:54|
|1     |223    |4.0   |2005-04-02 23:46:13|
|1     |253    |4.0   |2005-04-02 23:35:40|
|1     |260    |4.0   |2005-04-02 23:33:46|
+------+-------+------+-------------------+
only showing top 10 rows



validRatingsDF = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [4]:
// Take only the first 1000 entries for writing
val limitedRatingsDF = validRatingsDF.limit(1000)

limitedRatingsDF = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [5]:
// Write the DataFrame to HDFS, partitioned by userId
limitedRatingsDF.write
  .mode("overwrite")
  .partitionBy("userId") // Partition the data by userId
  .csv(outputPath)

println(s"Data written to $outputPath for 1000 valid entries. Verify the structure in HDFS.")

Data written to hdfs:///user/Day16AndDay17/case_study_2/user-data for 1000 valid entries. Verify the structure in HDFS.


In [6]:
// Stop the SparkSession
spark.stop()